In [32]:
import os
import pandas as pd
import json
import requests
from time import sleep
from dotenv import load_dotenv
import logging

load_dotenv()
api_key = os.getenv("API_KEY")
secret  = os.getenv("SECRET")

In [33]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("../logs/vessels.log"),  # Log to a file
        logging.StreamHandler()  # Log to the console
    ]
)

# Configure download directory
DOWNLOAD_DIR = os.path.abspath("../downloads/vessels")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [34]:
def to_df(json_data):
    data = json_data.get("data", [])
    new_df = pd.DataFrame(data)

    return new_df

In [35]:
def make_request(mmsi_list):
    url = f"https://api.myshiptracking.com/api/v2/vessel/bulk?mmsi={mmsi_list}&response=extended"

    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    response = requests.get(url, headers=headers)
    if not response.ok:
        logging.error(f"Error: {response.status_code}, {response.text}")

    return response.json()

In [36]:
port_calls = pd.read_csv("../data/port_calls.csv")
vessels_list = port_calls.mmsi.unique().tolist()

In [37]:
combined_df = pd.DataFrame()

In [47]:
# Split the vessels_list into chunks of 100 and join each chunk into a comma-separated string
for i in range(0, len(vessels_list), 100):
    chunk = vessels_list[i:i+100]
    mmsi_list = ",".join(map(str, chunk))

    data = make_request(mmsi_list)
    if data:
            new_df = to_df(data)
            combined_df = pd.concat([combined_df, new_df], ignore_index=True)
            combined_df.to_csv(os.path.join(DOWNLOAD_DIR, f"vessels_{i//100}.csv"), index=False)
    else:
        logging.warning(f"Chunk {chunk}: no data")
    
    sleep(0.5)

    logging.info(f"Chunk {chunk}: DONE")


2025-05-04 14:12:08,752 - INFO - Chunk [259739000, 563153900, 538009610, 636021492, 229865000, 538008166, 563674000, 647521310, 354423000, 356060000, 477325800, 257714000, 372028000, 417222405, 255806091, 311032200, 477833200, 538003850, 209312000, 563135900, 538003114, 563475000, 356505000, 311013600, 565893000, 229949000, 229819000, 574002770, 538006947, 255806249, 563154200, 351435000, 636014945, 352980874, 563026400, 256127000, 477455900, 371866000, 352002338, 372424000, 538010803, 538009659, 255806209, 563089500, 258731000, 636022611, 636016338, 256510000, 563178500, 538009878, 538003191, 477899800, 431839000, 538005436, 563084900, 538009376, 538010426, 305097000, 311001200, 353718000, 311000788, 566049000, 311907000, 477144800, 636023858, 354275000, 311000936, 256547000, 431934000, 257496000, 538002206, 564100000, 578001200, 538006239, 563108100, 210607000, 370357000, 352755000, 477842100, 431605000, 563087500, 538009750, 538006284, 414171000, 566204000, 352001780, 319091700, 538

In [45]:
vessels_list = [mmsi for mmsi in vessels_list if mmsi not in vessels_8['mmsi'].tolist()]